# 03 - Modelagem Analítica (Camada Gold – Data Warehouse)

## Objetivo da Camada Gold

A camada Gold tem como objetivo disponibilizar os dados em um formato analítico, otimizado para consultas e análises de negócio. Nesta etapa, os dados da camada Silver são organizados em um **Data Warehouse no modelo estrela**, composto por tabelas fato e dimensão.

Essa modelagem facilita a análise de indicadores relacionados à manutenção preditiva, permitindo responder às perguntas de negócio definidas no início do MVP.


In [0]:
# Leitura da camada Silver
df_silver = spark.table("maintenance_silver")

df_silver.show(5)
df_silver.printSchema()


+---+----------+----+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|udi|product_id|type|air_temperature_k|process_temperature_k|rotational_speed_rpm|torque_nm|tool_wear_min|machine_failure|twf|hdf|pwf|osf|rnf|
+---+----------+----+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|  1|    M14860|   M|            298.1|                308.6|                1551|     42.8|            0|              0|  0|  0|  0|  0|  0|
|  2|    L47181|   L|            298.2|                308.7|                1408|     46.3|            3|              0|  0|  0|  0|  0|  0|
|  3|    L47182|   L|            298.1|                308.5|                1498|     49.4|            5|              0|  0|  0|  0|  0|  0|
|  4|    L47183|   L|            298.2|                308.6|                1433|     39.5|            7|              0|  0|  0|  0|  0|  0|

## Dimensão Máquina

A dimensão máquina concentra atributos descritivos dos equipamentos, permitindo análises segmentadas por tipo de máquina. Embora o dataset possua informações limitadas sobre o equipamento, a criação desta dimensão é importante para demonstrar a modelagem dimensional e permitir expansões futuras do Data Warehouse.

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

dim_machine = df_silver.select(
    "product_id",
    "type"
).dropDuplicates()

dim_machine = dim_machine.withColumn(
    "machine_sk", monotonically_increasing_id()
)

dim_machine.show(5)


+----------+----+----------+
|product_id|type|machine_sk|
+----------+----+----------+
|    M14860|   M|         0|
|    L47181|   L|         1|
|    L47182|   L|         2|
|    L47183|   L|         3|
|    L47184|   L|         4|
+----------+----+----------+
only showing top 5 rows


A visualização inicial da tabela `dim_machine` confirma a criação correta da dimensão, contendo a chave substituta (`machine_sk`) e os atributos descritivos das máquinas, conforme a modelagem dimensional proposta.


In [0]:
dim_machine.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_machine")


A dimensão `dim_machine` foi persistida como tabela Delta gerenciada, garantindo armazenamento consistente, versionamento e disponibilidade para consultas analíticas e relacionamento com a tabela fato.


In [0]:
fact_maintenance = df_silver.join(
    dim_machine,
    on=["product_id", "type"],
    how="left"
).select(
    "machine_sk",
    "udi",
    "air_temperature_k",
    "process_temperature_k",
    "rotational_speed_rpm",
    "torque_nm",
    "tool_wear_min",
    "machine_failure",
    "twf",
    "hdf",
    "pwf",
    "osf",
    "rnf"
)

fact_maintenance.show(5)


+----------+---+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|machine_sk|udi|air_temperature_k|process_temperature_k|rotational_speed_rpm|torque_nm|tool_wear_min|machine_failure|twf|hdf|pwf|osf|rnf|
+----------+---+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|      5086|  1|            298.1|                308.6|                1551|     42.8|            0|              0|  0|  0|  0|  0|  0|
|      8636|  2|            298.2|                308.7|                1408|     46.3|            3|              0|  0|  0|  0|  0|  0|
|      9622|  3|            298.1|                308.5|                1498|     49.4|            5|              0|  0|  0|  0|  0|  0|
|      4361|  4|            298.2|                308.6|                1433|     39.5|            7|              0|  0|  0|  0|  0|  0|
|      3017|  5|            298.2|

A visualização inicial da tabela `fact_maintenance` confirma a criação da tabela fato, contendo as métricas operacionais e a chave estrangeira (`machine_sk`), em conformidade com o modelo estrela definido para o Data Warehouse.


In [0]:
fact_maintenance.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("fact_maintenance")


A tabela fato `fact_maintenance` foi persistida como tabela Delta gerenciada, consolidando os dados analíticos do Data Warehouse e viabilizando análises de falhas e métricas operacionais de forma estruturada.


##Insight
A integração entre as tabelas `dim_machine` e `fact_maintenance`, estruturadas em um modelo dimensional no formato estrela, possibilita análises analíticas robustas sobre o comportamento operacional dos equipamentos. A separação entre atributos descritivos das máquinas e métricas operacionais reduz redundâncias e facilita consultas eficientes e escaláveis.

As métricas registradas na tabela fato evidenciam o monitoramento contínuo de variáveis críticas, como temperatura do ar, temperatura do processo, velocidade de rotação, torque e desgaste da ferramenta, permitindo acompanhar o estado operacional dos equipamentos ao longo do tempo. A predominância de registros sem falha (`machine_failure = 0`) indica que a maior parte do conjunto de dados representa operação normal, cenário típico em ambientes industriais e que reforça a importância de análises mais refinadas para identificar padrões que antecedem falhas.

A presença do atributo `type` na dimensão de máquinas viabiliza a segmentação das análises por categoria de equipamento, permitindo avaliar se determinados tipos apresentam maior sensibilidade a condições operacionais específicas ou maior propensão a falhas. Além disso, a existência de indicadores específicos de falha (TWF, HDF, PWF, OSF e RNF) possibilita a identificação de diferentes mecanismos de falha e sua relação com as condições de operação.

Por fim, o uso de uma chave substituta (`machine_sk`) assegura rastreabilidade, governança e flexibilidade evolutiva do modelo analítico, tornando o Data Warehouse uma base adequada para análises de manutenção preditiva e suporte à tomada de decisão no contexto industrial.


## Modelo Estrela

O Data Warehouse foi modelado no formato estrela, composto por:

- **Tabela Fato:** `fact_maintenance`
  - Contém métricas operacionais e indicadores de falha
- **Dimensão:** `dim_machine`
  - Contém atributos descritivos da máquina (produto e tipo)

Essa estrutura permite análises eficientes e escaláveis sobre eventos de manutenção e falhas de equipamentos.


In [0]:
%sql
SELECT
  d.type,
  AVG(f.machine_failure) AS taxa_falha_media
FROM fact_maintenance f
JOIN dim_machine d
  ON f.machine_sk = d.machine_sk
GROUP BY d.type
ORDER BY taxa_falha_media DESC;


type,taxa_falha_media
L,0.03916666666666667
M,0.02769436102769436
H,0.020937188434695914


A análise da taxa média de falha segmentada por tipo de máquina evidencia diferenças relevantes no comportamento operacional dos equipamentos. As máquinas do tipo **L** apresentam a maior taxa média de falha, aproximadamente **3,9%**, seguidas pelas máquinas do tipo **M**, com cerca de **2,8%**, enquanto as máquinas do tipo **H** apresentam a menor taxa média de falha, em torno de **2,1%**.

Esse resultado sugere que máquinas do tipo **L** são mais suscetíveis à ocorrência de falhas quando comparadas aos demais tipos, o que pode estar relacionado a características específicas de operação, carga de trabalho ou condições de uso. Por outro lado, o menor índice observado nas máquinas do tipo **H** indica maior robustez ou maior tolerância às condições operacionais analisadas.

Esses achados reforçam a importância de estratégias de manutenção diferenciadas por tipo de equipamento, permitindo priorizar ações preventivas e preditivas nos grupos com maior propensão a falhas, otimizando recursos e reduzindo riscos operacionais.


In [0]:
%sql
SELECT
  machine_failure,
  AVG(air_temperature_k) AS avg_air_temp,
  AVG(process_temperature_k) AS avg_process_temp,
  AVG(rotational_speed_rpm) AS avg_rpm,
  AVG(torque_nm) AS avg_torque,
  AVG(tool_wear_min) AS avg_tool_wear
FROM fact_maintenance
GROUP BY machine_failure;


machine_failure,avg_air_temp,avg_process_temp,avg_rpm,avg_torque,avg_tool_wear
0,299.9739985508732,309.9955698167888,1540.2600144912535,39.62965531518476,106.69371700652107
1,300.886430678466,310.2902654867255,1496.486725663717,50.168141592920364,143.78171091445427


A comparação das médias das variáveis operacionais entre cenários de operação normal (`machine_failure = 0`) e de falha (`machine_failure = 1`) evidencia diferenças significativas no comportamento dos equipamentos. Observa-se que, nos registros com falha, as máquinas operam, em média, com temperaturas do ar e do processo ligeiramente mais elevadas, indicando maior estresse térmico durante esses períodos.

Além disso, os equipamentos que apresentaram falha operam com menor velocidade média de rotação, porém com valores substancialmente mais altos de torque, o que sugere maior esforço mecânico aplicado durante a operação. O desgaste médio da ferramenta também é consideravelmente superior nos cenários de falha, reforçando a relação direta entre desgaste acumulado e a ocorrência de falhas.

Esses resultados indicam que combinações de maior desgaste da ferramenta, aumento de torque e elevação de temperaturas constituem condições operacionais críticas, capazes de aumentar a probabilidade de falhas. Dessa forma, o monitoramento contínuo dessas variáveis mostra-se fundamental para estratégias de manutenção preditiva, permitindo a identificação antecipada de padrões que precedem falhas e a adoção de ações preventivas.


In [0]:
%sql
SELECT
  SUM(twf) AS total_twf,
  SUM(hdf) AS total_hdf,
  SUM(pwf) AS total_pwf,
  SUM(osf) AS total_osf,
  SUM(rnf) AS total_rnf
FROM fact_maintenance;


total_twf,total_hdf,total_pwf,total_osf,total_rnf
46,115,95,98,19


A análise da distribuição dos tipos de falha revela diferenças relevantes na frequência dos mecanismos que afetam os equipamentos. As falhas relacionadas à dissipação de calor (**HDF**) apresentam a maior incidência, totalizando **115 ocorrências**, seguidas pelas falhas por sobrecarga (**OSF**), com **98 registros**, e pelas falhas relacionadas à potência (**PWF**), com **95 ocorrências**.

As falhas associadas ao desgaste da ferramenta (**TWF**) aparecem com menor frequência relativa, totalizando **46 casos**, enquanto as falhas aleatórias (**RNF**) são as menos recorrentes, com apenas **19 registros**. Esse comportamento indica que a maior parte das falhas observadas está associada a fatores operacionais específicos e mensuráveis, e não a eventos aleatórios.

Esses resultados reforçam a importância do monitoramento de variáveis térmicas, mecânicas e elétricas, uma vez que falhas relacionadas à dissipação de calor, sobrecarga e potência representam a maior parcela dos eventos registrados. Dessa forma, estratégias de manutenção preditiva devem priorizar esses mecanismos de falha, permitindo ações preventivas mais eficazes e direcionadas.


## Discussão dos Resultados

As análises realizadas a partir da camada Gold indicam que condições operacionais mais severas — como aumento de temperaturas, maior desgaste da ferramenta e elevação do torque — estão associadas a maiores taxas de falha. Observa-se também comportamento distinto entre os tipos de máquinas, evidenciando a importância da segmentação por categoria de equipamento.

Esses resultados demonstram que a modelagem dimensional adotada viabiliza análises analíticas consistentes e fornece uma base adequada para aplicações de manutenção preditiva e suporte à tomada de decisão.



## Catálogo de Dados – Camada Gold

### Dimensão: dim_machine

| Coluna | Tipo | Descrição | Domínio / Observações |
|------|------|-----------|----------------------|
| machine_sk | Integer | Chave substituta da máquina | Gerada automaticamente |
| product_id | String | Identificador do produto | Valores alfanuméricos |
| type | String | Tipo da máquina | L, M, H |


### Fato: fact_maintenance

| Coluna | Tipo | Descrição | Domínio / Observações |
|------|------|-----------|----------------------|
| udi | Integer | Identificador único do registro | Valor sequencial |
| machine_sk | Integer | Chave da dimensão máquina | FK para dim_machine |
| air_temperature_k | Double | Temperatura do ar (K) | ~295 a 305 |
| process_temperature_k | Double | Temperatura do processo (K) | ~305 a 315 |
| rotational_speed_rpm | Integer | Velocidade de rotação | Valores positivos |
| torque_nm | Double | Torque aplicado | Valores positivos |
| tool_wear_min | Integer | Desgaste da ferramenta | ≥ 0 |
| machine_failure | Integer | Indicador de falha | 0 = não, 1 = sim |
| twf | Integer | Falha por desgaste | 0 ou 1 |
| hdf | Integer | Falha por dissipação de calor | 0 ou 1 |
| pwf | Integer | Falha de potência | 0 ou 1 |
| osf | Integer | Falha por sobrecarga | 0 ou 1 |
| rnf | Integer | Falha aleatória | 0 ou 1 |


In [0]:
%sql
SELECT COUNT(*) FROM fact_maintenance;


COUNT(*)
10000


In [0]:
%sql
SELECT COUNT(*) FROM dim_machine;


COUNT(*)
10000


Ao final da construção da camada Gold, foi realizada a verificação do número de registros na tabela fato, confirmando a manutenção das **10.000 instâncias** esperadas. Esse resultado assegura que a modelagem dimensional e as transformações aplicadas não ocasionaram perda de dados, validando a consistência do Data Warehouse para as análises subsequentes.
